In [1]:
#import needed libraries 
from urllib.request import urlretrieve
import zipfile, os
import time, sys, copy
import pandas as pd
import scipy.sparse as sps
import numpy as np
from collections import defaultdict
import math

In [2]:
#clone repositeries
!git clone https://github.com/shalini1194/RKT
!git clone https://github.com/lyf-1/PEBG.git
    
#copy repositories in working directory
!cp -r ./RKT/* ./
#!cp -r ./PEBG/assist09/* ./
#!cp -r ../input/skillbuilder-data-2009-2010/2012-2013-data-with-predictions-4-final.csv ./

Cloning into 'RKT'...
remote: Enumerating objects: 53, done.
remote: Counting objects: 100% (53/53), done.
remote: Compressing objects: 100% (42/42), done.
remote: Total 53 (delta 9), reused 44 (delta 7), pack-reused 0
Unpacking objects: 100% (53/53), done.
Cloning into 'PEBG'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 50 (delta 0), reused 1 (delta 0), pack-reused 47
Unpacking objects: 100% (50/50), done.


In [3]:
#import from github cloned repositories
from RKT import utils
import RKT.model_rkt as model_rkt


In [ ]:
#Passages needed to recompute pro_pro_skills
#Initially I will use file with the ones already computed by RKT authors
#In the future I will modify the code and try to compute better ones.

""" 
import os 
import pandas as pd
import numpy as np
from scipy import sparse


class DataProcess():
    def __init__(self, data_folder='assist09', file_name='skill_builder_data_corrected_collapsed.csv', min_inter_num=3):
        print("Process Dataset %s" % data_folder)
        self.min_inter_num = min_inter_num
        self.data_folder = data_folder
        self.file_name = file_name

    def process_csv(self):
        #pre-process original csv file for assist dataset

        # read csv file
        data_path = os.path.join(self.data_folder, self.file_name)
        df = pd.read_csv(data_path, low_memory=False, encoding="ISO-8859-1")
        print('original records number %d' % len(df))

        # delete empty skill_id
        df = df.dropna(subset=['skill_id'])
        df = df[~df['skill_id'].isin(['noskill'])]
        print('After removing empty skill_id, records number %d' % len(df))

        # delete scaffolding problems
        df = df[df['original'].isin([1])]
        print('After removing scaffolding problems, records number %d' % len(df))

        #delete the users whose interaction number is less than min_inter_num
        users = df.groupby(['user_id'], as_index=True)
        delete_users = []
        for u in users:
            if len(u[1]) < self.min_inter_num:
                delete_users.append(u[0])
        print('deleted user number based min-inters %d' % len(delete_users))
        df = df[~df['user_id'].isin(delete_users)]
        print('After deleting some users, records number %d' % len(df))
        # print('features: ', df['assistment_id'].unique(), df['answer_type'].unique())

        df.to_csv(os.path.join(self.data_folder, '%s_processed.csv'%self.file_name))


    def pro_skill_graph(self):
        df = pd.read_csv(os.path.join(self.data_folder, '%s_processed.csv'%self.file_name),low_memory=False, encoding="ISO-8859-1")
        problems = df['problem_id'].unique()
        pro_id_dict = dict(zip(problems, range(len(problems))))
        print('problem number %d' % len(problems))

        pro_type = df['problem_type'].unique()
        pro_type_dict = dict(zip(pro_type, range(len(pro_type))))
        print('problem type: ', pro_type_dict)

        pro_feat = []
        pro_skill_adj = []
        skill_id_dict, skill_cnt = {}, 0
        for pro_id in range(len(problems)):            
            tmp_df = df[df['problem_id']==problems[pro_id]]
            tmp_df_0 = tmp_df.iloc[0]

            # pro_feature: [ms_of_response, answer_type, mean_correct_num]
            ms = tmp_df['ms_first_response'].abs().mean()
            p = tmp_df['correct'].mean()
            pro_type_id = pro_type_dict[tmp_df_0['problem_type']] 
            tmp_pro_feat = [0.] * (len(pro_type_dict)+2)
            tmp_pro_feat[0] = ms
            tmp_pro_feat[pro_type_id+1] = 1.
            tmp_pro_feat[-1] = p
            pro_feat.append(tmp_pro_feat)

            # build problem-skill bipartite
            tmp_skills =[]
            for tmp_df_0 in tmp_df:
                tmp_skills.append(tmp.tmp_df_0)
            for s in tmp_skills:
                if s not in skill_id_dict:
                    skill_id_dict[s] = skill_cnt
                    skill_cnt += 1
                pro_skill_adj.append([pro_id, skill_id_dict[s], 1])

        pro_skill_adj = np.array(pro_skill_adj).astype(np.int32)
        pro_feat = np.array(pro_feat).astype(np.float32)
        pro_feat[:, 0] = (pro_feat[:, 0] - np.min(pro_feat[:, 0])) / (np.max(pro_feat[:, 0])-np.min(pro_feat[:, 0]))
        pro_num = np.max(pro_skill_adj[:, 0]) + 1
        skill_num = np.max(pro_skill_adj[:, 1]) + 1
        print('problem number %d, skill number %d' % (pro_num, skill_num))

        # save pro-skill-graph in sparse matrix form
        pro_skill_sparse = sparse.coo_matrix((pro_skill_adj[:, 2].astype(np.float32), (pro_skill_adj[:, 0], pro_skill_adj[:, 1])), shape=(pro_num, skill_num))
        sparse.save_npz(os.path.join(self.data_folder, 'pro_skill_sparse.npz'), pro_skill_sparse)

        # take joint skill as a new skill
        skills = df['skill_id'].unique()
        for s in skills:
            if '_' in s:
                skill_id_dict[s] = skill_cnt
                skill_cnt += 1 

        # save pro-id-dict, skill-id-dict
        self.save_dict(pro_id_dict, os.path.join(self.data_folder, 'pro_id_dict.txt'))
        self.save_dict(skill_id_dict, os.path.join(self.data_folder, 'skill_id_dict.txt'))

        # save pro_feat_arr
        np.savez(os.path.join(self.data_folder, 'pro_feat.npz'), pro_feat=pro_feat)

    def generate_user_sequence(self, seq_file):
        # generate user interaction sequence
        # and write to data.txt

        df = pd.read_csv(os.path.join(self.data_folder, '%s_processed.csv'%self.file_name), low_memory=False, encoding="ISO-8859-1")
        ui_df = df.groupby(['user_id'], as_index=True)   
        print('user number %d' % len(ui_df))

        user_inters = []
        cnt = 0
        for ui in ui_df:
            tmp_user, tmp_inter = ui[0], ui[1]
            tmp_problems = list(tmp_inter['problem_id'])
            tmp_skills = list(tmp_inter['skill_id'])
            tmp_ans = list(tmp_inter['correct'])
            tmp_end_time = list(tmp_inter['end_time'])
            user_inters.append([[len(tmp_inter)], tmp_skills, tmp_problems, tmp_ans, tmp_end_time])
        
        write_file = os.path.join(self.data_folder, seq_file)
        self.write_txt(write_file, user_inters)


    def save_dict(self, dict_name, file_name):
        f = open(file_name, 'w')
        f.write(str(dict_name))
        f.close


    def write_txt(self, file, data):
        with open(file, 'w') as f:
            for dd in data:
                for d in dd:
                    f.write(str(d)+'\n')


    def read_user_sequence(self, filename, max_len=200, min_len=3, shuffle_flag=True):
        with open(filename, 'r') as f:
            lines = f.readlines()
        with open(os.path.join(self.data_folder, 'skill_id_dict.txt'), 'r') as f:
            skill_id_dict = eval(f.read()) 
        with open(os.path.join(self.data_folder, 'pro_id_dict.txt'), 'r') as f:
            pro_id_dict = eval(f.read())
        

        y, skill, problem, real_len, timestamp = [], [], [], [], []
        skill_num, pro_num = len(skill_id_dict), len(pro_id_dict)
        print('skill num, pro num, ', skill_num, pro_num)

        index = 0
        while index < len(lines):
            num = eval(lines[index])[0]
            tmp_skills = eval(lines[index+1])[:max_len]
            # tmp_skills = [skill_id_dict[ele]+1 for ele in tmp_skills]     # for assist09
            tmp_skills = [ele+1 for ele in tmp_skills]                      # for assist12 
            tmp_pro = eval(lines[index+2])[:max_len]
            tmp_pro = [pro_id_dict[ele]+1 for ele in tmp_pro]
            tmp_ans = eval(lines[index+3])[:max_len]
            tmp_time = eval(lines[index+4])[:max_len]

            if num>=min_len:
                tmp_real_len = len(tmp_skills)
                # Completion sequence
                tmp_ans += [-1]*(max_len-tmp_real_len)
                tmp_skills += [0]*(max_len-tmp_real_len)
                tmp_pro += [0]*(max_len-tmp_real_len)
                tmp_time += [-1]*(max_len-tmp_real_len)

                y.append(tmp_ans)
                skill.append(tmp_skills)
                problem.append(tmp_pro)
                real_len.append(tmp_real_len)
                timestamp

            index += 5
        
        y = np.array(y).astype(np.float32)
        skill = np.array(skill).astype(np.int32)
        problem = np.array(problem).astype(np.int32)
        real_len = np.array(real_len).astype(np.int32)

        print(skill.shape, problem.shape, y.shape, real_len.shape)      
        print(np.max(y), np.min(y))
        print(np.max(real_len), np.min(real_len))  
        print(np.max(skill), np.min(skill))
        print(np.max(problem), np.min(problem))

        np.savez(os.path.join(self.data_folder, "%s.npz"%self.file_name), problem=problem, y=y, skill=skill, real_len=real_len, skill_num=skill_num, problem_num=pro_num)



data_folder = './'
min_inter_num = 3
file_name='2012-2013-data-with-predictions-4-final.csv'
DP = DataProcess(data_folder, file_name, min_inter_num)

DP.process_csv()
DP.pro_skill_graph()
DP.generate_user_sequence('data.txt')
DP.read_user_sequence(os.path.join(data_folder, 'data.txt')) """



In [4]:
#Code from RKT train with few changes for performance

import argparse
import psutil
import gc
import pandas as pd
from random import shuffle
from sklearn.metrics import roc_auc_score, accuracy_score
from scipy import sparse
import torch.nn as nn
from torch.optim import Adam
from torch.nn.utils import clip_grad_norm_
from torch.nn.utils.rnn import pad_sequence
from collections import  defaultdict
from sys import getsizeof

from RKT.model_rkt import RKT
from RKT.utils import *

start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

print(torch.cuda.is_available())


def compute_corr(prob_seq, next_seq, corr_dic):
    corr= np.zeros((prob_seq.shape[0],prob_seq.shape[1], prob_seq.shape[1]))
    for i in range(0,prob_seq.shape[0]):
        for  j in range(0,next_seq.shape[1] ):
            for k in range(j+1):
                corr[i][j][k]=corr_dic[next_seq[i][j]][prob_seq[i][k]]
    return corr
def get_data(df, file2, max_length, train_split=0.8, randomize=True):
    """Extract sequences from dataframe.
    Arguments:
        df (pandas Dataframe): output by prepare_data.py
        max_length (int): maximum length of a sequence chunk
        train_split (float): proportion of data to use for training
    """

    pro_pro_sparse = sparse.load_npz('./pro_pro_sparse.npz')

    pro_pro_coo = pro_pro_sparse.tocoo()
    # print(pro_skill_csr)
    pro_pro_dense = pro_pro_coo.toarray()


    item_ids = [torch.tensor(u_df["item_id"].values, dtype=torch.long)
                for _, u_df in df.groupby("user_id")]
    # skill_ids = [torch.tensor(u_df["skill_id"].values, dtype=torch.long)
    #              for _, u_df in df.groupby("user_id")]
    labels = [torch.tensor(u_df["correct"].values, dtype=torch.long)
              for _, u_df in df.groupby("user_id")]



    item_inputs = [torch.cat((torch.zeros(1, dtype=torch.long), i + 1))[:-1] for i in item_ids]
    # skill_inputs = [torch.cat((torch.zeros(1, dtype=torch.long), s + 1))[:-1] for s in skill_ids]
    label_inputs = [torch.cat((torch.zeros(1, dtype=torch.long), l))[:-1] for l in labels]

    def chunk(list):
        if list[0] is None:
            return list
        list = [torch.split(elem, max_length) for elem in list]
        return [elem for sublist in list for elem in sublist]

    # Chunk sequences
    lists = (item_inputs, label_inputs, item_ids, labels)
    chunked_lists = [chunk(l) for l in lists]

    data = list(zip(*chunked_lists))
    if randomize:
        shuffle(data)

    # Train-test split across users
    train_size = int(train_split * len(data))
    train_data, val_data = data[:train_size], data[train_size:]
    return pro_pro_dense, train_data, val_data


def prepare_batches(train_data, batch_size, randomize=True):
    """Prepare batches grouping padded sequences.
    Arguments:
        data (list of lists of torch Tensor): output by get_data
        batch_size (int): number of sequences per batch
    Output:
        batches (list of lists of torch Tensor)
    """
    process = psutil.Process(os.getpid())
    print('PB entered: ', process.memory_info().rss)
    # if randomize:
    #     shuffle(train_data)
    batches = []
    train_y, train_skill, train_problem, timestamps, train_real_len = train_data[0], train_data[1], train_data[2], train_data[3], train_data[4]
    print('train_y created: ', process.memory_info().rss)
    gc.collect()
    print('train_data deleted: ', process.memory_info().rss)
    item_ids = [torch.LongTensor(i).cuda() for i in train_problem]
    print('train_data deleted: ', process.memory_info().rss)
    timestamp = [torch.LongTensor(timestamp).cuda() for timestamp in timestamps]
    print('train_data deleted: ', process.memory_info().rss)
    labels = [torch.LongTensor(i).cuda() for i in train_y]
    print('train_data deleted: ', process.memory_info().rss)
    item_inputs = [torch.cat((torch.zeros(1, dtype=torch.long).cuda(), i))[:-1] for i in item_ids]
    print('train_data deleted: ', process.memory_info().rss)
    # skill_inputs = [torch.cat((torch.zeros(1, dtype=torch.long), s))[:-1] for s in skill_ids]
    label_inputs = [torch.cat((torch.zeros(1, dtype=torch.long).cuda(), l))[:-1] for l in labels]
    print('train_data deleted: ', process.memory_info().rss)
    data = list(zip(item_inputs, label_inputs, item_ids, timestamp, labels))
    print(getsizeof(data))
    print(len(data))
    print(batch_size)
    print(getsizeof(item_inputs))
    print(getsizeof(label_inputs))
    print(getsizeof(item_ids))
    print('data created: ', process.memory_info().rss)
    del train_y
    del train_skill
    del train_problem
    del timestamps
    del train_data
    del train_real_len
    del item_inputs
    del label_inputs
    del item_ids
    del timestamp
    del labels
    gc.collect(0)
    print(gc.garbage)
    print('train_y deleted: ', process.memory_info().rss)
    for k in range(0, len(data), batch_size):
        print(k)
        batch = data[k:k + batch_size]
        seq_lists = list(zip(*batch))
        
        inputs_and_ids = [pad_sequence(seqs, batch_first=True, padding_value=0)
                          for seqs in seq_lists[:-1]]
        labels = pad_sequence(seq_lists[-1], batch_first=True, padding_value=-1)  # Pad labels with -1
        batches.append([*inputs_and_ids, labels])
    return batches


def train_test_split(data, split=0.8):
    n_samples = data[0].shape[0]
    split_point = int(n_samples*split)
    train_data, test_data = [], []
    for d in data:
        train_data.append(d[:split_point])
        test_data.append(d[split_point:])
    del data
    return train_data, test_data


def compute_auc(preds, labels):
    preds = preds[labels >= 0].flatten()
    labels = labels[labels >= 0].float()
    if len(torch.unique(labels)) == 1:  # Only one class
        auc = accuracy_score(labels, preds.round())
        acc = auc
    else:
        auc = roc_auc_score(labels, preds)
        acc = accuracy_score(labels, preds.round())
    return auc, acc


def compute_loss(preds, labels, criterion):
    preds = preds[labels >= 0].flatten()
    labels = labels[labels >= 0].float()
    return criterion(preds, labels)
def computeRePos(time_seq, time_span):
    batch_size = time_seq.shape[0]
    size = time_seq.shape[1]

    time_matrix= (torch.abs(torch.unsqueeze(time_seq, axis=1).repeat(1,size,1).reshape((batch_size, size*size,1)) - \
                 torch.unsqueeze(time_seq,axis=-1).repeat(1, 1, size,).reshape((batch_size, size*size,1))))

    # time_matrix[time_matrix>time_span] = time_span
    time_matrix = time_matrix.reshape((batch_size,size,size))


    return (time_matrix)
def get_corr_data(pro_num):
    pro_pro_file = pd.read_csv("../input/ednet-dataset/ednet_corr.csv",names=["pro1", "pro2", "corr"], dtype={"row":int, "col":int, "corr":float})
    pro1List = pro_pro_file.pro1.tolist()
    pro2List = pro_pro_file.pro2.tolist()
    corrList = pro_pro_file["corr"].tolist()
    print(pro_pro_file.memory_usage().sum() / 1024 / 1024)
    pro_pro_sparse = sps.coo_matrix((corrList, (pro1List, pro2List)))
    pro_pro_sparse = pro_pro_sparse.tocsr()
    del (pro_pro_file, pro1List, pro2List, corrList)
    return pro_pro_sparse


def train(train_data, val_data, pro_num, corr_data, timestamp, timespan,  model, optimizer, logger, saver, num_epochs, batch_size, grad_clip):
    """Train SAKT model.
    Arguments:
        train_data (list of tuples of torch Tensor)
        val_data (list of tuples of torch Tensor)
        model (torch Module)
        optimizer (torch optimizer)
        logger: wrapper for TensorboardX logger
        saver: wrapper for torch saving
        num_epochs (int): number of epochs to train for
        batch_size (int)
        grad_clip (float): max norm of the gradients
    """
    process = psutil.Process(os.getpid())
    print('entered train', process.memory_info().rss)
    criterion = nn.BCEWithLogitsLoss()
    step = 0
    metrics = Metrics()
    print('PB memory used: ', process.memory_info().rss)
    
    for epoch in range(num_epochs):
        print("in epoch"+str(epoch))
        print("Prepare batches train")
        train_batches = prepare_batches(train_data, batch_size)
        gc.collect()
        print("Prepare batches val")
        val_batches = prepare_batches(val_data, batch_size)
        gc.collect()
        # Training
        i=0
        for item_inputs, label_inputs, item_ids, timestamp, labels in train_batches:
            # rel = compute_corr(item_inputs, item_ids, corr_data)
            print(i)
            rel = corr_data[(item_ids-1).cpu().unsqueeze(1).repeat(1,item_ids.cup().shape[-1],1),(item_inputs.cup()-1).unsqueeze(-1).repeat(1,1,item_inputs.cpu().shape[-1])]
            item_inputs = item_inputs.cuda()
            time = computeRePos(timestamp, timespan)
            # skill_inputs = skill_inputs.cuda()
            label_inputs = label_inputs.cuda()
            item_ids = item_ids.cuda()
            # skill_ids = skill_ids.cuda()
            preds, weights = model(item_inputs, label_inputs, item_ids, torch.Tensor(rel).cuda(), time.cuda())

            loss = compute_loss(preds, labels.cuda(), criterion)
            preds = torch.sigmoid(preds).detach().cpu()
            train_auc, train_acc = compute_auc(preds, labels)
            model.zero_grad()
            loss.backward()
            clip_grad_norm_(model.parameters(), grad_clip)
            optimizer.step()
            step += 1
            metrics.store({'loss/train': loss.item()})
            metrics.store({'auc/train': train_auc})

            # Logging
            if step == len(train_batches)-1:
                torch.save(weights, 'weight_tensor_rel')
            # print(step)
            if step % 1000 == 0:
                print(metrics.average())
                print(step)

                # weights = {"weight/" + name: param for name, param in model.named_parameters()}
                # grads = {"grad/" + name: param.grad
                #         for name, param in model.named_parameters() if param.grad is not None}
                # logger.log_histograms(weights, step)
                # logger.log_histograms(grads, step)
            i+=1
            if i%1000==0 :
                gc.collect()
            

        # Validation

        model.eval()
        for item_inputs, label_inputs, item_ids, timestamp, labels in val_batches:
            rel = corr_data[(item_ids - 1).unsqueeze(1).repeat(1, item_ids.shape[-1], 1), (item_inputs - 1).unsqueeze(-1).repeat(1,1, item_inputs.shape[-1])]
            item_inputs = item_inputs.cuda()
            # skill_inputs = skill_inputs.cuda()
            time = computeRePos(timestamp, timespan)
            label_inputs = label_inputs.cuda()
            item_ids = item_ids.cuda()
            # skill_ids = skill_ids.cuda()
            with torch.no_grad():
                preds,weights = model(item_inputs, label_inputs, item_ids, torch.Tensor(rel).cuda(), time.cuda())

                preds = torch.sigmoid(preds).cpu()


            val_auc, val_acc = compute_auc(preds, labels)
            metrics.store({'auc/val': val_auc, 'acc/val': val_acc})
            gc.collect()
        model.train()

        # Save model

        average_metrics = metrics.average()
        logger.log_scalars(average_metrics, step)
        print(average_metrics)
        stop = saver.save(average_metrics['auc/val'], model)
        if stop:
            break


parser = argparse.ArgumentParser(description='Train RKT.')
parser.add_argument('--dataset', type=str)
parser.add_argument('--logdir', type=str, default='runs/rkt')
parser.add_argument('--savedir', type=str, default='save/rkt')
parser.add_argument('--max_length', type=int, default=200)
parser.add_argument('--embed_size', type=int, default=200)
parser.add_argument('--num_attn_layers', type=int, default=1)
parser.add_argument('--num_heads', type=int, default=5)
parser.add_argument('--encode_pos', action='store_true')
parser.add_argument('--max_pos', type=int, default=10)
parser.add_argument('--drop_prob', type=float, default=0.2)
parser.add_argument('--batch_size', type=int, default=1024)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--grad_clip', type=float, default=10)
parser.add_argument('--num_epochs', type=int, default=20)
parser.add_argument('--timespan', default=100000, type=int)
args2 = {"batch_size": 50,
         "train_steps": 1000,
         "logdir": 'runs/rkt',
         "savedir": 'save/rkt',
         "max_length": 200,
         "embed_size": 200,
         "num_attn_layers": 1,
         "num_heads": 5,
         "encode_pos": True,
         "max_pos": 10,
         "drop_prob": 0.2,
         "batch_size": 200,
         "lr": 1e-3,
         "grad_clip":10.0,
         "num_epochs": 300,
         "timespan": 100000}

args = parser.parse_args(args=[])

# full_df = pd.read_csv('./', sep=",")
# train_df = pd.read_csv('../../KT-GAT/data/ed_net2_train.csv', sep=",")
# test_df = pd.read_csv('../../KT-GAT/data/ed_net2_test.csv', sep=",")
# # train_data_file = '../KT-GAT/data/ed_net.csv'
# corr_dic, train_data, val_data = get_data(train_df, '../../KT-GAT/RKT/ednet_corr', args.max_length)
# print(len(train_data))
process = psutil.Process(os.getpid())
gc.enable()
data = np.load('../input/ednet-dataset/ednet.npz')
print('df created', process.memory_info().rss)

y, skill, problem, timestamp, real_len = data['y'], data['skill'], data['problem'], data['time'] , data['real_len']
skill_num, pro_num = data['skill_num'], data['problem_num']
print('problem number %d, skill number %d' % (pro_num, skill_num))
print('before data delete', process.memory_info().rss)
del data
gc.collect()
print(gc.garbage)
print('after data delete', process.memory_info().rss)
corr_data = get_corr_data(pro_num)
print('coor_data created', process.memory_info().rss)
# divide train test set
train_data, test_data = train_test_split([y, skill, problem, timestamp, real_len])
print('y,skill,problem created', process.memory_info().rss)
del (y, skill, problem)
print(gc.garbage)
gc.collect()
print('y, skill, problem deleted', process.memory_info().rss)
num_items = pro_num
# num_items = int(full_df["item_id"].max() + 1)
# num_skills = int(full_df["skill_id"].max() + 1)

model = RKT(num_items, args.embed_size, args.num_attn_layers, args.num_heads,
              args.encode_pos, args.max_pos, args.drop_prob).cuda()
optimizer = Adam(model.parameters(), lr=args.lr)

# Reduce batch size until it fits on GPU
while True:
    #try:
        # Train
    param_str = (f'{args.dataset},'
                 f'batch_size={args.max_length},'
                 f'max_length={args.max_length},'
                 f'encode_pos={args.encode_pos},'
                 f'max_pos={args.max_pos}')
    logger = Logger(os.path.join(args.logdir, param_str))
    saver = Saver(args.savedir, param_str)
    print('before train', process.memory_info().rss)
    train(train_data, test_data, pro_num, corr_data, timestamp, args.timespan, model, optimizer, logger, saver, args.num_epochs,
          args.batch_size, args.grad_clip)
    break
    #except RuntimeError:
     #   args.batch_size = args.batch_size // 2
      #  print(RuntimeError)
       # print(f'Batch does not fit on gpu, reducing size to {args.batch_size}')

logger.close()

param_str = (f'{args.dataset},'
              f'batch_size={args.batch_size},'
              f'max_length={args.max_length},'
              f'encode_pos={args.encode_pos},'
              f'max_pos={args.max_pos}')
saver = Saver(args.savedir, param_str)
model = saver.load()
# test_data, _ = get_data(test_df, args.max_length, train_split=1.0, randomize=False)
test_batches = prepare_batches(test_data, args.batch_size, randomize=False)
corr_data = get_corr_data(pro_num)
test_preds = np.empty(0)

# Predict on test set
print("pre eval")
model.eval()
print("post eval")
correct = np.empty(0)
for item_inputs, label_inputs, item_ids, labels in test_batches:
    print("in loop")
    rel = corr_data[
        (item_ids - 1).unsqueeze(1).repeat(1, item_ids.shape[-1], 1), (item_inputs - 1).unsqueeze(-1).repeat(1, 1, item_inputs.shape[ -1])]
    item_inputs = item_inputs.cuda()
    # skill_inputs = skill_inputs.cuda()
    label_inputs = label_inputs.cuda()
    item_ids = item_ids.cuda()
    # skill_ids = skill_ids.cuda()
    with torch.no_grad():
        preds = model(item_inputs, label_inputs, item_ids)
        preds = torch.sigmoid(preds[labels >= 0]).flatten().cpu().numpy()
        test_preds = np.concatenate([test_preds, preds])
    labels = labels[labels>=0].float()
    correct = np.concatenate([correct, labels])
    print(correct)


print(correct.shape)
print(test_preds.shape)
print("auc_test = ", roc_auc_score(correct, test_preds))
print("acc_test = ", accuracy_score(correct, test_preds))

True
df created 353980416
problem number 12372, skill number 1901
before data delete 3298570240
[]
after data delete 3298570240
617.0325241088867
coor_data created 3624140800
y,skill,problem created 3624140800
[]
y, skill, problem deleted 3624140800
before train 5173755904
entered train 5173755904
PB memory used:  5173755904
in epoch0
Prepare batches train
PB entered:  5173755904
train_y created:  5173755904
train_data deleted:  5173755904
train_data deleted:  5596889088
train_data deleted:  6026342400
train_data deleted:  6438612992
train_data deleted:  7170994176
train_data deleted:  7867719680
5150408
588157
1024
4826320
4826320
4826320
data created:  7929626624
[]
train_y deleted:  7929626624
0
1024
2048
3072
4096
5120
6144
7168
8192
9216
10240
11264
12288
13312
14336
15360
16384
17408
18432
19456
20480
21504
22528
23552
24576
25600
26624
27648
28672
29696
30720
31744
32768
33792
34816
35840
36864
37888
38912
39936
40960
41984
43008
44032
45056
46080
47104
48128
49152
50176
51200
5

TypeError: can't convert cuda:0 device type tensor to numpy. Use Tensor.cpu() to copy the tensor to host memory first.

In [ ]:
def data_partition(data):
    usernum = 0
    itemnum = 0
    User = defaultdict(list)
    user_train = {}
    user_valid = {}
    user_test = {}
    # assume user/item index starting from 1
    for index, row in data.iterrows():
        #print(row)
        u, i, order = row['user_id'], row['problem_id'], row['order_id']
        if not(math.isnan(float(u)) or math.isnan(float(i)) or math.isnan(float(order))):
            u = int(u)
            i = int(i)
            order = int(order)
            usernum = max(u, usernum)
            itemnum = max(i, itemnum)
            User[u].append((i, order))

    for user in User:
        nfeedback = len(User[user])
        if nfeedback < 3:
            user_train[user] = User[user]
            user_valid[user] = []
            user_test[user] = []
        else:
            user_train[user] = User[user][:-2]
            user_valid[user] = []
            user_valid[user].append(User[user][-2])
            user_test[user] = []
            user_test[user].append(User[user][-1])
    return [user_train, user_valid, user_test, usernum, itemnum]

data_assestments = pd.read_csv('../input/assestments-2009-2010/non_skill_builder_data_new.csv', header=0, error_bad_lines=False, warn_bad_lines=True)

partition = data_partition(data_assestments)
print(partition[0])

In [ ]:
print(data[0])